<a href="https://colab.research.google.com/github/imratnesh/llm_parse_agent/blob/main/llm_finetune_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")

In [ ]:
df = pd.DataFrame(dataset['train'])
df.head()

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2",
                                                      num_labels=3)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
!pip install evaluate
import evaluate


In [ ]:

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

# Configure training arguments
training_args = TrainingArguments(
   output_dir="test_trainer", # Directory to save the model checkpoints and logs
   num_train_epochs=1,          # Set the number of training epochs to 1
   #evaluation_strategy="epoch", # Strategy for evaluation (e.g., evaluate at the end of each epoch)
   per_device_train_batch_size=1,  # Batch size per device during training
   per_device_eval_batch_size=1,    # Batch size per device during evaluation
   gradient_accumulation_steps=4 # Number of updates steps to accumulate the gradients for
)

# Initialize the Trainer with the model, training arguments, datasets, and compute_metrics function
trainer = Trainer(
   model=model,

   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

# Start training the model
trainer.train()


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=250, training_loss=1.1424254150390625, metrics={'train_runtime': 16024.9654, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.016, 'total_flos': 522598219776000.0, 'train_loss': 1.1424254150390625, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.9187292456626892,
 'eval_accuracy': 0.559,
 'eval_runtime': 4040.4124,
 'eval_samples_per_second': 0.247,
 'eval_steps_per_second': 0.247,
 'epoch': 1.0}

Choosing between fine-tuning and RAG
When deciding whether to use fine-tuning or RAG, consider the following factors:

Nature of the task: For tasks that benefit from highly specialized models (e.g., domain-specific applications), fine-tuning is often the preferred approach. RAG is ideal for tasks that require integration of external knowledge or real-time information retrieval.

Data availability: Fine-tuning requires a substantial amount of labeled data specific to the task. If such data is scarce, RAG’s retrieval component can compensate by providing relevant information from external sources.

Resource constraints: Fine-tuning can be computationally intensive, whereas RAG leverages existing databases to supplement the generative model, potentially reducing the need for extensive training.
